By - Dipanshu Kumar

# **Instructions to Run the Notebook**

Follow the steps below to execute this notebook successfully:


**Step 1: Setup Environment**


*   This notebook uses Google Colab’s T4 GPU for faster execution. Ensure that the runtime is set to GPU.
*   **Go to Runtime > Change runtime type > Select GPU.**



**Step 2: Install & Import Dependencies**


*   Run the first few cells to install necessary packages (if required).
*   Import the required libraries before executing the main code.


**Step 3: Initialize the GenAI Model (Ollama)**


*   The system uses Llama 3.1 via ChatOllama. Ensure that the model is correctly loaded before proceeding.
*   Have provided the info to download and run Llama3.1 later in the notebook


**Step 4: Execute the Main Loop**


*   Run the main() function to start movement detection and advisory feedback.
*   **The system will:**

      *   Continuously and randomly collect sensor data (Acceleration, GPS speed, Temperature).
      *   Classify movement as Walking, Running, or Stepping on Stairs.

      *   Generate a human-like advisory response based on age, gender, weight, and environmental conditions.
      *   Convert the response into speech output at regular intervals.

**Important Points to Consider:**

*   In this notebook, **random data** is used to simulate sensor readings for acceleration, GPS speed, and temperature because the real sensor data from an actual smartphone cannot be accessed directly within this environment. This is a placeholder for the actual sensor input in a real-world application.
*   In a real implementation on a smartphone, we could replace the random data generation with actual sensor inputs using the following APIs that I have found from the internet: SensorManager API,CoreMotion API,LocationManager API,CoreLocation APIs etc.


*   Regular intervals for feedback are set using **time.sleep(50)**. We can modify this for a different update frequency.
*   If any error occurs, check the exception messages printed in the console and re-run the affected cells.

In case of any query please reach out to me.

In [16]:
!pip install pyttsx3

In [17]:
!pip install langchain

In [18]:
!pip install -qU langchain-ollama

In [19]:
!pip install colab-xterm

In [20]:
!pip install gTTS

In [21]:
# Import required libraries

import time  # For time-based operations like pausing the script
import os  # To handle file operations (e.g., removing files)
import gtts  # Google Text-to-Speech for converting text to speech
import random  # For generating random data (e.g., sensor data)
import ollama  # Library to interact with Ollama's LLM models
import IPython.display as ipd  # To display audio player in Colab

# Import Langchain's ChatOllama, which integrates with Ollama's language model
from langchain_ollama import ChatOllama

In [22]:

def speak(text):

    """Convert text to speech using gTTS, play it in Colab, and delete the file."""
    audio_file = "advice.mp3"

    try:
        # Convert text to speech
        tts = gtts.gTTS(text)
        tts.save(audio_file)

        # Play the audio in Colab
        audio = ipd.Audio(audio_file, autoplay=True)
        display(audio)

        # Wait for the audio to play before deleting the file
        import time
        time.sleep(3)  # Adjust based on length of speech

        # Remove the audio file after playing
        os.remove(audio_file)

    except Exception as e:
        print(f"An error occurred: {e}")


In [23]:

def get_sensor_data():

    """Simulate accelerometer, GPS, and temperature data."""

    try:
        # Generate simulated accelerometer data (in m/s²)
        acceleration = random.uniform(0.5, 3.0)

        # Generate simulated GPS speed data (in m/s)
        gps_speed = random.uniform(0, 3.0)

        # Generate simulated temperature data (in Celsius) and round to 2 decimal places
        temperature = round(random.uniform(10, 40), 2)

        return acceleration, gps_speed, temperature

    except Exception as e:
        print(f"An error occurred while generating sensor data: {e}")
        return None  # Return None in case of an error


In [24]:

def detect_movement(acceleration, gps_speed):

    """Determine the movement type based on acceleration and GPS speed."""

    try:
        # Check if input values are valid numbers
        if not isinstance(acceleration, (int, float)) or not isinstance(gps_speed, (int, float)):
            raise ValueError("Acceleration and GPS speed must be numeric values.")

        # Determine the movement type based on thresholds
        if gps_speed > 2.0 and acceleration > 1.5:
            return "running"
        elif 0.5 < gps_speed <= 2.0 and acceleration > 0.8:
            return "walking"
        elif acceleration > 2.5:
            return "stepping on stairs"
        else:
            return "stationary"

    except ValueError as ve:
        print(f"Invalid input: {ve}")
        return None  # Return None in case of invalid input
    except Exception as e:
        print(f"An error occurred while detecting movement: {e}")
        return None


In [25]:

def generate_advice(movement, age, gender, weight, temperature, llm):

    """Generate a human-like response using Ollama's LLM."""

    try:
        # Construct the prompt with user details
        prompt = f"""
        A {age}-year-old {gender} weighing {weight} kg is {movement}.
        The temperature is {temperature}°C.
        Provide a human-like response about the safety of this activity.
        Your output will be in following structure:

        Temperature : {temperature},
        Age : {age},
        Weight : {weight},
        Movement : {movement},
        Response : Your Suggestion.
        """

        # Use Ollama's LLM to generate a response
        response = llm.invoke(prompt)
        response = response.content  # Extract content from response

        # Print and return the generated advice
        print(response)
        return response

    except Exception as e:
        # Handle any errors that occur during execution
        print(f"An error occurred: {e}")
        return None  # Return None if an error happens


In [26]:

def main():

    """Main loop to detect movement and provide feedback."""

    age, gender, weight = 30, "male", 70  # Simulated user profile

    while True:
        try:
            # Get sensor data (acceleration, GPS speed, temperature)
            acceleration, gps_speed, temperature = get_sensor_data()

            # Detect movement based on sensor data
            movement = detect_movement(acceleration, gps_speed)

            # Generate advice based on detected movement and user profile
            advice = generate_advice(movement, age, gender, weight, temperature, llm)

            # Speak the generated advice
            speak(advice)

            # Wait before the next iteration
            time.sleep(50)  # Provide feedback every 50 seconds

        except Exception as e:
            # Handle any unexpected errors in the loop
            print(f"An error occurred in the main loop: {e}")


To use the Ollama LLaMA 3.1 8B model, follow the detailed steps below:

**Run the next cell, you will be directed to Xterm :-**

1. **Install Ollama**: Start by running the following command in your terminal. This will download and install Ollama on your machine:

*   curl -fsSL https://ollama.com/install.sh | sh

  This script will automatically fetch and install the Ollama package, setting it up for use. You will be prompted to provide appropriate permissions during installation if required.

2. **Start the Ollama Service**: Once the installation is complete, you need to initiate the Ollama service. This will enable Ollama to run locally and serve models for processing. Run the following command:

*   ollama serve &

  The "&" allows the server to run in the background, freeing up the terminal for other commands.

3. **Run the LLaMA 3.1 8B Model**: After the Ollama service is up, you can now install and run the LLaMA 3.1 8B model by executing:

*   ollama run llama3.1

  This will install the LLaMA 3.1 8B model if it hasn't been installed yet. The model will then be loaded and ready to process inputs for inference or other tasks

By following these steps, you'll have Ollama set up on your local machine with the LLaMA 3.1 8B model.


For more details follow the link : https://github.com/ollama/ollama

In [40]:
# Load the colabxterm extension in Google Colab, which allows access to a terminal interface
%load_ext colabxterm

# Open an xterm terminal within the Colab environment, providing a command-line interface
%xterm

The colabxterm extension is already loaded. To reload it, use:
  %reload_ext colabxterm


Launching Xterm...

<IPython.core.display.Javascript object>

In [41]:
# Initialize the ChatOllama model with the specified version
llm = ChatOllama(model="llama3.1")

In [ ]:
# Call the main function to start the loop for detecting movement and providing feedback
main()

Temperature : 28.48,
Age : 30,
Weight : 70,
Movement : walking,
Response : It seems like a perfect day for a walk! The temperature is mild, and there's no apparent health concern based on your age or weight. Just remember to stay hydrated by drinking plenty of water before, during, and after your walk. Wear comfortable shoes and clothes suitable for the weather. Enjoy your stroll!


Temperature : 11.56,
Age : 30,
Weight : 70,
Movement : stationary,
Response : Given the relatively mild temperature of 11.56°C, I'd say this activity seems safe for a healthy individual like yourself. However, as you're stationary, please make sure to stay hydrated and avoid any potential cold-related health issues by dressing warmly and staying indoors if possible. Just to be on the safe side!


Temperature : 38.28,
Age : 30,
Weight : 70,
Movement : stepping on stairs,
Response : It's quite hot today, so I'd suggest taking it easy while walking up the stairs to avoid overheating and dehydration. Maybe try to take breaks in between flights to catch your breath and cool down for a bit? And don't forget to stay hydrated by drinking plenty of water throughout the day!


### **Why chosen Llama3.1 8B:**

**Open-Source and Efficient**: Llama 3.1 8B is an open-source model, designed for environments requiring quick and efficient performance without heavy resource demands.

**Versatile Performance**: The model excels in tasks such as text summarization, text classification, sentiment analysis, and language translation, all with low-latency inferencing.

**Compact but Powerful**: Despite its smaller size, Llama 3.1 8B is well-suited for applications like chatbots and basic content creation, offering robust natural language understanding and generation capabilities.